# Sveltish
> A Python package that ~~kind of~~ implements Svelte Stores.


`Svelte Stores` are one of the secret weapons of the [Svelte framework](https://svelte.dev/) (the recently voted [most loved web framework](https://insights.stackoverflow.com/survey/2021#section-most-loved-dreaded-and-wanted-web-frameworks)). 

Stores allow easy [reactive programming](https://en.wikipedia.org/wiki/Reactive_programming) by presenting an [Observer pattern](https://en.wikipedia.org/wiki/Observer_pattern) that is as simple as necessary, but not simpler.

## Install

```sh
pip install sveltish
```

## How to use

Sometimes, you'll have values that need to be accessed by multiple unrelated objects.

For that, you can use `stores`.  It is a very simple implementation (around 100 lines of code) of the Observer/Observable pattern. 

A store is simply an object with a `subscribe` method that allows interested parties to be notified when its value changes. 

#### __Writable Stores__ 

In [ ]:
from sveltish.stores import writable

In [ ]:
#|hide
from nbdev import show_doc
from fastcore.test import test_eq, test_fail

In [ ]:
#|hide
show_doc(writable)

---

[source](https://github.com/fredguth/sveltish/blob/main/sveltish/stores.py#LNone){target="_blank" style="float:right; font-size:smaller"}

### writable

>      writable (value:~T=None, start:Callable[[Callable[[~T],NoneType]],Optiona
>                l[Callable[[],NoneType]]]=<function noop>)

Creates a new Writable Store (A Writable factory).

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| value | T | None | initial value of the store |
| start | Notifier | noop | Optional Notifier, a function called when the first subscriber is added |
| **Returns** | **Writable[T]** |  | **Writable Store** |

In [ ]:
count = writable(0)
history = []  # logging for testing
# subscribe returns an unsubscriber
def record(x): 
    history.append(x)
    print(history)
stop = count.subscribe(record)

test_eq(history, [0])

[0]


We just created a `count` store. Its value can be accessed via a `callback` we pass in the `count.subscribe` method:

A __Writable__ can be set from the outside. When it happens, all its subscribers will react.

In [ ]:
def increment(): count.update(lambda x: x + 1)
def decrement(): count.update(lambda x: x - 1)
def reset(): count.set(0)

count.set(3)
increment()
decrement()
decrement()
reset()
count.set(42)

test_eq(history, [0, 3, 4, 3, 2, 0, 42])

[0, 3]
[0, 3, 4]
[0, 3, 4, 3]
[0, 3, 4, 3, 2]
[0, 3, 4, 3, 2, 0]
[0, 3, 4, 3, 2, 0, 42]


The `unsubscriber`, in this example the `stop` function, stops the notifications to the `subscriber`. 

In [ ]:
stop()
reset()
count.set(22)
test_eq(history, [0, 3, 4, 3, 2, 0, 42])
count

w$int <0>:22

Notice that you can still change the `store` but there was no print message this time.  There was no observer listening. 

:::{.callout-note}
`Observer`, `Subscriber` and `Callback` are used as synomyms here.
:::

When we subscribe new callbacks, they will be promptly informed of the current state of the `store`.

In [ ]:
stop  = count.subscribe(lambda x: print(f"Count is now {x}"))
stop2 = count.subscribe(lambda x: print(f"double of count is {2*x}"))

Count is now 22
double of count is 44


In [ ]:
reset()

Count is now 0
double of count is 0


In [ ]:
stop()
stop2()

You can create an empty `Writable Store`.

In [ ]:
store = writable()
history = []
unsubscribe = store.subscribe(lambda x: history.append(x))
unsubscribe()
test_eq(history, [None])

If you try to unsubscribe twice, it won't break.  It just does nothing the second time... and in the third time... and...

In [ ]:
unsubscribe(), unsubscribe(), unsubscribe()

(None, None, None)

Stores assume mutable objects. 

::: {.callout-note}
In Python everythong is an object.  Here we are calling an object something that is not a primitive (eg. int, bool, etc)
:::

In [ ]:
class Bunch:
    __init__ = lambda self, **kw: setattr(self, '__dict__', kw)

obj = Bunch()
called = 0
store = writable(obj)
def callback(x):
    global called
    called += 1
stop = store.subscribe(callback)

In [ ]:
test_eq(called, 1)
obj.a = 1 #type: ignore
store.set(obj)
test_eq(called, 2)

#### __Readable Stores__

However... It is clear that not all stores should be writable by whoever has a reference to them. Many times you want a single `publisher` of change in store that is only consumed (`subscribed`) by many other objects. For those cases, we have readable stores.

:::{.callout-note}
The `Publisher Subscriber (PubSub)` pattern is a variant of the `Observable/Observer` pattern.
:::

In [ ]:
from sveltish.stores import readable

In [ ]:
#|hide
show_doc(readable)

---

[source](https://github.com/fredguth/sveltish/blob/main/sveltish/stores.py#LNone){target="_blank" style="float:right; font-size:smaller"}

### readable

>      readable (value:~T, start:Callable[[Callable[[~T],NoneType]],Optional[Cal
>                lable[[],NoneType]]])

Creates a new Readable Store (A Readable factory).

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| value | T | initial value of the store |
| start | Notifier | function called when the first subscriber is added |
| **Returns** | **Readable[T]** | **Readable Store** |

A Readable store without a `start` function is a constant value and has no meaning for us. Therefore, `start` is a required argument.

In [ ]:
try:
    c = readable(0) # shoud fail
except Exception as error:
    print(error)

test_fail(lambda: readable(0))

readable() missing 1 required positional argument: 'start'


In [ ]:
class Publisher:
    def __init__(self): self.set = lambda x: None
    def set_set(self, set): 
        self.set = set
        return lambda: None
    def use_set(self, value): self.set(value)

In [ ]:
p = Publisher()
reader = readable(0, p.set_set)
reader

r$int <0>:0

Ths store only starts updating after the first subscriber. Here, the publisher does not change the store.

In [ ]:
p.use_set(1), reader

(None, r$int <0>:0)

In [ ]:
stop = reader.subscribe(lambda x: print(f"reader is now {x}"))

reader is now 0


In [ ]:
p.use_set(2)

reader is now 2


In [ ]:
stop()

Another example of Readable Store usage:

In [ ]:
from threading import Event, Thread
import time

In [ ]:
def start(set): # the start function is the publisher
    stopped = Event()
    def loop(): # needs to be in a separate thread
        while not stopped.wait(1): # in seconds
            set(time.localtime())
    Thread(target=loop).start()    
    return stopped.set

In [ ]:
now = readable(time.localtime(), start)
now

r$struct_time <0>:time.struct_time(tm_year=2023, tm_mon=3, tm_mday=8, tm_hour=10, tm_min=1, tm_sec=59, tm_wday=2, tm_yday=67, tm_isdst=0)

:::{.callout-note}
The `loop` needs to be in its own thread, otherwise the function would never return and we would wait forever.
:::

While there is no subscriber, the Readable will not be updated.

In [ ]:
now

r$struct_time <0>:time.struct_time(tm_year=2023, tm_mon=3, tm_mday=8, tm_hour=10, tm_min=1, tm_sec=59, tm_wday=2, tm_yday=67, tm_isdst=0)

In [ ]:
OhPleaseStop = now.subscribe(lambda x: print(time.strftime(f"%H:%M:%S", x), end="\r"))

In [ ]:
time.sleep(2)
OhPleaseStop()

::: {.callout-note} 

The Svelte Store api allow you to create a Readable Store without a Notifier. See discussion [here.](https://github.com/sveltejs/svelte/issues/8300)

:::

#### __Derived Stores__

A `Derived Store` stores a value based on the value of another store.

In [ ]:
from sveltish.stores import derived

In [ ]:
#|hide
show_doc(derived)

---

[source](https://github.com/fredguth/sveltish/blob/main/sveltish/stores.py#LNone){target="_blank" style="float:right; font-size:smaller"}

### derived



Creates a new Derived Store (A Derived factory).

For example:

In [ ]:
count = writable(1)
stopCount = count.subscribe(lambda x: print(f"count is {x}"))
double = derived(count, lambda x: x * 2)
stopDouble = double.subscribe(lambda x: print(f"double is {x}"))
test_eq(double.get(), 2*count.get())

count is 1
double is 2


In [ ]:
count.set(2)
test_eq(double.get(), 4)

double is 4
count is 2


In [ ]:
stopCount(), stopDouble()

(None, None)

Building on our previous example, we can create a store that derives the elapsed time since the original store was started.

In [ ]:
elapsing = None
def calc_elapsed(now):
    global elapsing
    if not elapsing: 
        elapsing = now
    return time.mktime(now) - time.mktime(elapsing)

In [ ]:
now

r$struct_time <0>:time.struct_time(tm_year=2023, tm_mon=3, tm_mday=8, tm_hour=10, tm_min=2, tm_sec=2, tm_wday=2, tm_yday=67, tm_isdst=0)

In [ ]:
elapsed = derived(now, lambda x: calc_elapsed(x))
elapsed

r$float <0>:0.0

In [ ]:
stopElapsed = elapsed.subscribe(lambda x: print(f"Elapsed time of source store: {x} seconds.", end="\r"))

In [ ]:
time.sleep(1)
stopElapsed()

Derived stores allow us to transform the value of a store. In RxPy they are called `operators`. You can build several operators like: `filter`, `fold`, `map`, `zip`...

Let's build a custom `filter` operator:

In [ ]:
user = writable({"name": "John", "age": 32})
stopLog = user.subscribe(lambda x: print(f"User: {x}"))

User: {'name': 'John', 'age': 32}


In [ ]:
name = derived(user, lambda x: x["name"])
stopName = name.subscribe(lambda x: print(f"Name: {x}"))

Name: John


In [ ]:
user.update(lambda x: x | {"age": 45})

User: {'name': 'John', 'age': 45}


Updating the age does not trigger the `name subscriber`. Let's see what happens when we update the name.

In [ ]:
user.update(lambda x: x | {"name": "Fred"})

User: {'name': 'Fred', 'age': 45}
Name: Fred


Only changes to the name of the user triggers the `name` subscriber.

In [ ]:
stopName(), stopLog()

(None, None)

Another cool thing about Derived Stores is that you can derive from a list of stores. Let's build a `zip` operator.

In [ ]:
a = writable([1,2,3,4])
b = writable([5,6,7,8])
a,b

(w$list <0>:[1, 2, 3, 4], w$list <0>:[5, 6, 7, 8])

In [ ]:
zipper = derived([a,b], lambda a,b: list(zip(a,b)))

In [ ]:
test_eq(zipper.get(), [(1, 5), (2, 6), (3, 7), (4, 8)])

While `zipper` has no subscribers, it keeps the initial value, it is `stopped`.  

In [ ]:
a.set([4,3,2,1])
test_eq(zipper.get(), [(1, 5), (2, 6), (3, 7), (4, 8)])

A subscription `starts` zipper and it will start to react to the changes of the stores.

In [ ]:
u = zipper.subscribe(lambda x: None)
test_eq(zipper.get(), [(4, 5), (3, 6), (2, 7), (1, 8)])

In [ ]:
b.set([8,7,6,5])
test_eq(zipper.get(), [(4, 8), (3, 7), (2, 6), (1, 5)])

In [ ]:
u()

#### Store composition with pipes

In [ ]:
writable(1).pipe(lambda x: x + 1).pipe(lambda x: x * 2)

r$int <0>:4

In [ ]:
writable(1).pipe(lambda x: x+1, lambda x: x*2)

r$int <0>:4

In [ ]:
writable(1) | (lambda x: x+1) | (lambda x: x*2) 

r$int <0>:4

In [ ]:
a = writable(1)
u5 = (a 
      | (lambda x: x*2) 
      | (lambda x: x*2) 
      | (lambda x: x*2)).subscribe(lambda x: print(f"u5: {x}"))

u5: 8


In [ ]:
a.set(2)

u5: 16


In [ ]:
u5()

## Missing features

You may have noticed that along the way we had always to subscribe and then had to remember to unsubscribe when we were done. This is a bit of a nuisance. 
Svelte has a compiler that provide some [syntatic sugar](https://svelte.dev/tutorial/auto-subscriptions) to make this easier. They call it `auto-subscriptions`.

`Sveltish` does not have `auto-subscriptions` yet. But if you have a nice idea how to implement it, please let me know.